In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#import libraries
import pandas as pd


In [ ]:
df=pd.read_csv('../input/emailspamoriginal/spam.csv',names=['output','input'])
df.head()

In [ ]:
# STEP1 =  CLEANING
# REMOVE STOPWORD, stemming, lemmatization, convert to lowercase

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

from nltk.stem import WordNetLemmatizer 
from nltk.tokenize import word_tokenize

import re
lemmatizer = WordNetLemmatizer() 

In [ ]:


train=[]

for i in range(0,len(df)):
    preprocess=re.sub('[^a-zA-Z0-9]',' ',df['input'][i]) #remove all the junk except alphanumeric
    preprocess=preprocess.lower()   #convert everything to lowercase
    preprocess=word_tokenize(preprocess) # break each sentence into words
    #apply lemmatization to only those words who are not included in stopwords
    preprocess=[lemmatizer.lemmatize(word) for word in preprocess if not word in stopwords.words('english')]  
    
    #join each word by spaces
    
    preprocess=' '.join(preprocess)
    #append to a list as 'preprocess' will iterate through whole loop and only give the last sentence 
    train.append(preprocess)



In [ ]:
#train

In [ ]:
# BAG OF WORDS = Represent each sentence as a array of binary numbers

from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=3000) #out of 8086 features, we choose  the top 3000 features 
X = cv.fit_transform(train).toarray()

In [ ]:
X.shape
# since the  shape is (5574,8046), this means that every sentence is represented by 8046 features or numbers which is very long
# hence we used the max_features variable

In [ ]:
# now we devide the hamm and spam into dummy variable
y=pd.get_dummies(df['output'])

#to avoid the curse of dimetionality (means hmara kaam 1 column se bhi ban rha tha instead of 2)
y=y.iloc[:,1].values # we only select 1 column


In [ ]:
# Train Test Split

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [ ]:
#use multinomial naive bayes
from sklearn.naive_bayes import MultinomialNB
spam_detect_model = MultinomialNB().fit(X_train, y_train)

y_pred=spam_detect_model.predict(X_test)


In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

In [ ]:
y_test.shape


In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred, normalize=False)